## Import libraries

In [ ]:
import sys
import requests
from haqs_api import haqs_api

NoneType = type(None)

%load_ext autoreload
%autoreload 2

## Data addresses

1. **Stations request** http://api.gios.gov.pl/pjp-api/rest/station/findAll
2. **Sensors request** http://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId}
3. **Data request** http://api.gios.gov.pl/pjp-api/rest/data/getData/{sensorId}
4. **AQ index request** http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/{stationId}

# Geopandas part

## Get Stations from GIOŚ API

In [3]:
stations_json = haqs_api.get_stations()
print("Number of available stations: ", len(stations_json))

Number of available stations:  160


In [4]:
stations_df = haqs_api.create_stations_gdf(stations_json)
stations_df.head()

,station_id,geometry
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## Plot Stations on map

In [5]:
haqs_api.create_stations_map(stations_json)

## Get list of Sensors for each Station

In [6]:
sensors_df = haqs_api.create_sensors_df(stations_json)
sensors_df.head()

,station_id,sensor_id,parameter
0,114,642,NO2
1,114,644,O3
2,117,660,CO
3,117,14395,PM10
4,117,658,C6H6


## Get Sensors readings from GIOŚ API

In [7]:
sensors_df = haqs_api.get_latest_sensors_readings(sensors_df)
sensors_df.head()

,station_id,sensor_id,parameter,data
0,114,642,NO2,17.74120
1,114,644,O3,127.73800
2,117,660,CO,432.76100
3,117,14395,PM10,27.34230
4,117,658,C6H6,0.00069


In [8]:
available_parameters = haqs_api.get_available_parameters(sensors_df)
available_parameters

['NO2', 'O3', 'CO', 'PM10', 'C6H6', 'PM2.5', 'SO2']

## Visualize readings

In [9]:
params_df = haqs_api.get_param_df(stations_df, sensors_df, 'PM10')
haqs_api.show_readings_map(params_df)

Loading BokehJS ...

# Connect with database

In [123]:
conn = haqs_api.connect_with_db()

Successfully connected with DataBase!


In [122]:
haqs_api.close_db_connection(conn)

Connection has been closed!


## Create PostGIS Extension

In [124]:
haqs_api.create_postgis_extension(conn)

SAVEPOINT created!
extension "postgis" already exists

Rollback to SAVEPOINT!


## <font color="blue">Station Table</font>

### Create Stations Table

In [125]:
haqs_api.create_stations_table(conn)

SAVEPOINT created!
relation "stations" already exists

Rollback to SAVEPOINT!


### Insert Stations into DataBase

In [16]:
stations_json = haqs_api.get_stations()

In [20]:
stations_json[0]

{'id': 114,
 'stationName': 'Wrocław - Bartnicza',
 'gegrLat': '51.115933',
 'gegrLon': '17.141125',
 'city': {'id': 1064,
  'name': 'Wrocław',
  'commune': {'communeName': 'Wrocław',
   'districtName': 'Wrocław',
   'provinceName': 'DOLNOŚLĄSKIE'}},
 'addressStreet': 'ul. Bartnicza'}

In [126]:
# iterate over stations
for station in stations_json:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    try:
        haqs_api.db_insert_station(conn, station_id, station_lon, station_lat)
    except Exception as e:
        print(e)
        print ("Cannot execute insertion for Station ID: {}".format(station_id))

SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(114) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(117) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(129) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(52) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(109) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(11) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pk

SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(225) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(142) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(143) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(164) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(190) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "stations_pkey"
DETAIL:  Key (station_id)=(232) already exists.

Rollback to SAVEPOINT!


### Show Insertions

In [127]:
haqs_api.show_insertions(conn, "stations")

SAVEPOINT created!
SQL was successfully executed!


,station_id,geom
0,114,0101000020E6100000E3A59BC4202431407AC37DE4D68E...
1,117,0101000020E6100000736891ED7C073140346953758F90...
2,129,0101000020E6100000DA0418963F0331407E8CB96B098B...
3,52,0101000020E6100000BB809719362E30408A7780272D9A...
4,109,0101000020E6100000C07B478D094530404F3E3DB66562...
5,11,0101000020E6100000C746205ED79F2E4074B515FBCB74...
6,14,0101000020E6100000F4346090F4E12D400EA0DFF76F7C...
7,16,0101000020E61000003D2CD49AE6A53040F6798CF2CC5D...
8,38,0101000020E61000000FB4024356A73040C005D9B27C37...
9,67,0101000020E6100000C9570229B18330400C3F389F3A4A...


### Create Stations DataFrame from Stations Table

In [132]:
stations_gdf = haqs_api.return_stations_gdf(conn)
stations_gdf.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## <font color="blue">Sensors Table</font>

### Create Sensors Table

In [129]:
haqs_api.create_sensors_table(conn)

SAVEPOINT created!
relation "sensors" already exists

Rollback to SAVEPOINT!


### Insert Sensors into DataBase

> <i>**INSERT INTO**</i> sensors(sensor_id, sensor_parameter, station_id) **VALUES** (2, 'PM10', 114));

In [130]:
# iterate over stations
for station in stations_json:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    sensors_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    # iterate over station sensors
    for sensor in sensors_json:
        
        sensor_id = sensor["id"]
        sensor_parameter = sensor["param"]["paramCode"]
    
        try:
            haqs_api.db_insert_sensor(conn, sensor_id, sensor_parameter, station_id)
        except Exception as e:
            print(e)
            print ("Cannot execute insertion for Sensor ID: {} (Station ID: {})".format(sensor_id, station_id))

SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(642) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(644) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(660) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(14395) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(658) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(665) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAI

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16422) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16419) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16393) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16421) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16420) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(952) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (se

SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16041) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16043) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17232) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17239) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17235) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17240) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_p

SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16174) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16178) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16181) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16180) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16177) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(6062) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pk

SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(6396) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(6238) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(6236) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(6240) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(6212) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(6214) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
D

...

SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2770) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17896) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2766) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2772) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2768) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2774) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"


SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4701) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4706) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4703) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4707) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4709) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4715) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
D

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(15246) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(3127) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(3188) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(3195) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16706) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(3191) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique co

SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16046) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17225) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16048) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16044) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16050) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16051) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_p

SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1416) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1426) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1414) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1421) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1429) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1534) already exists.

Rollback to SAVEPOINT!
SAVEPOINT created!
duplicate key value violates unique constraint "sensors_pkey"
D

### Create Sensors DataFrame from Sensors Table

In [134]:
sensors_df = haqs_api.return_sensors_df(conn)
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


## <font color="blue">Readings Table</font>

### Create Readings Table  

In [135]:
haqs_api.create_readings_table(conn)

SAVEPOINT created!
relation "readings" already exists

Rollback to SAVEPOINT!


### Create list of available Sensors

In [138]:
sensors_ids = haqs_api.return_sensors_ids(conn)
sensors_ids[:2]

[642, 644]

### Insert Readings Into DataBase

#### Insert all available non NULL readings into Readings Table

In [ ]:
for sensor_id in sensors_ids:

    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    data = data_json['values']
    
    # check all available data
    for row in data:

        date = row['date']
        reading = row['value']

        # if there is no value wait until next attempt
        if isinstance(reading, NoneType):
            continue

        try:
            haqs_api.db_insert_sensor_readings(conn, sensor_id, date, reading, date, sensor_id)
        except Exception as e:
            print(e)

### Create Readings DataFrame from Readings Table

In [143]:
readings_df = haqs_api.return_readings_df(conn)
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [144]:
readings_df.describe()

,sensor_id,reading
count,55540.000000,55540.000000
mean,6858.797389,65.935307
std,6040.473448,136.320278
min,50.000000,0.000000
25%,2242.000000,4.737412
50%,4723.000000,18.450000
75%,14618.000000,44.282600
max,19816.000000,2928.870000


In [145]:
readings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55540 entries, 0 to 55539
Data columns (total 3 columns):
sensor_id    55540 non-null int64
date         55540 non-null object
reading      55540 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.3+ MB


## Create and save GeoDataFrame to Shapefile

In [57]:
stations_df.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


In [58]:
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


In [59]:
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [60]:
sensors_df = sensors_df.merge(stations_df, on='station_id')
sensors_df.head()

,sensor_id,sensor_parameter,station_id,geom
0,642,NO2,114,POINT (17.141125 51.115933)
1,644,O3,114,POINT (17.141125 51.115933)
2,660,CO,117,POINT (17.02925 51.129378)
3,14395,PM10,117,POINT (17.02925 51.129378)
4,658,C6H6,117,POINT (17.02925 51.129378)


In [61]:
readings_df = gpd.GeoDataFrame(readings_df.merge(sensors_df, on='sensor_id'))
readings_df.crs = from_epsg(4326)
readings_df.rename(index=str, columns={"geom": "geometry"}, inplace=True)
readings_df.head()

,sensor_id,date,reading,sensor_parameter,station_id,geometry
0,642,2018-09-07 18:00:00,12.93730,NO2,114,POINT (17.141125 51.115933)
1,642,2018-09-07 17:00:00,12.83960,NO2,114,POINT (17.141125 51.115933)
2,642,2018-09-07 16:00:00,11.44120,NO2,114,POINT (17.141125 51.115933)
3,642,2018-09-07 15:00:00,9.43838,NO2,114,POINT (17.141125 51.115933)
4,642,2018-09-07 14:00:00,10.22020,NO2,114,POINT (17.141125 51.115933)


In [62]:
readings_df.to_file('shp/readings.shp')